This notebook implement Text-To-Code Search Engine by code summarization method with `CodeT5` model: create code summarizations for all functions in database, then create embeddings summarizations with  `SentenceTransformers` model and calculate the NL-NL cosine similarity.

### Download test repositories and run `inspect4py` on them

In [1]:
# Repository picked from https://github.com as an example
repo = 'keon/algorithms'

In [2]:
!inspect4py --version

inspect4py, version 0.0.6


In [3]:
!mkdir -p content/output
%cd content/

!mkdir -p {repo} && git clone {f"https://github.com/{repo}.git"} {repo}
!inspect4py -i {repo} -o output/{repo} -sc -rm
%cd ..

/cs/home/cd271/Documents/Project/Examples/RepoAnalysis/SemanticCodeSearch/Text2code/content
Cloning into 'keon/algorithms'...
remote: Enumerating objects: 5162, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 5162 (delta 11), reused 16 (delta 3), pack-reused 5136
Receiving objects: 100% (5162/5162), 1.42 MiB | 10.99 MiB/s, done.
Resolving deltas: 100% (3231/3231), done.
Updating files: 100% (477/477), done.
Creating jsDir:output/keon/algorithms/algorithms/json_files
Creating jsDir:output/keon/algorithms/algorithms/algorithms/json_files
Creating jsDir:output/keon/algorithms/algorithms/algorithms/streaming/json_files
Creating jsDir:output/keon/algorithms/algorithms/algorithms/map/json_files
Error when processing separate_chaining_hashtable.py:  <class 'AttributeError'>
Error when processing hashtable.py:  <class 'AttributeError'>
Creating jsDir:output/keon/algorithms/algorithms/algorithms/stack/json_files
Error when proce

### Extract docstrings and functions from repositories.

In [2]:
import json

def funcs_to_lists(funcs, func_codes, docs):
    for func_name, func_info in funcs.items():
        if func_info.get("source_code") is not None:
            func_codes.append(func_info["source_code"])
        if func_info.get("doc") is None:
            continue
        for key in ["full", "long_description", "short_description"]:
            if func_info["doc"].get(key) is not None:
                docs.append(f"{func_name} {func_info['doc'].get(key)}")
                break

def file_to_lists(filename):
    func_codes = []
    docs = []
    with open(filename, "r") as f:
        dic = json.load(f)
    dic.pop("readme_files", None)
    for dir_name, files in dic.items():
        for file in files:
            if file.get("functions") is not None:
                funcs_to_lists(file["functions"], func_codes, docs)
            if file.get("classes") is not None:
                for class_name, class_info in file["classes"].items():
                    if class_info.get("methods") is not None:
                        funcs_to_lists(class_info["methods"], func_codes, docs)
    return func_codes, docs

In [3]:
repo_info = {}
function_list, _ = file_to_lists(f"content/output/{repo}/directory_info.json")

### Using fine-tuned `CodeT5` model from huggingface

In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/codet5-base-multi-sum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/cs/home/cd271/codesearch/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def get_code_sum(funcs):
    inputs = tokenizer.batch_encode_plus(
        funcs,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    # Perform inference to get code similarity
    with torch.no_grad():
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        outputs = model.generate(**inputs)
        
    similar_code_snippets = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return similar_code_snippets

In [6]:
from tqdm import tqdm

# Obtain function_list summarizations 
code_sum = []
with tqdm(total=len(function_list), desc="Generating code summaries") as pbar:
    for funcs in function_list:
        code_snippets = get_code_sum([funcs])
        code_sum.extend(code_snippets)
        pbar.update(1)

Generating code summaries:   0%|                                                               | 0/1171 [00:00<?, ?it/s]/cs/home/cd271/codesearch/lib64/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating code summaries: 100%|████████████████████████████████████████████████████| 1171/1171 [05:36<00:00,  3.48it/s]


In [7]:
# Example for input query
query = ['function to calculate cosine similarity']

In [8]:
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sum_model = SentenceTransformer("all-mpnet-base-v2", device=device)

def get_embedding(code_sum, query_sum):
    return sum_model.encode(code_sum, convert_to_tensor=True), sum_model.encode(query, convert_to_tensor=True)
    
code_sum_embeddings, query_sum_embedding = get_embedding(code_sum, query)

In [9]:
from torch.nn import CosineSimilarity

cosine_sim = CosineSimilarity(dim=1)
similarities = cosine_sim(query_sum_embedding, code_sum_embeddings).tolist()

In [10]:
def find_top_n_index(lst, n):
    largest_indices = []
    for i in range(n):
        max_value = max(lst)
        max_index = lst.index(max_value)
        largest_indices.append(max_index)
        lst[max_index] = float('-inf')
    return largest_indices

In [11]:
sim = similarities.copy()
index = find_top_n_index(sim,5)
print('Similiar code snippet:\n')
for i in index:
    print(f'Similarity: {similarities[i]}, \n{function_list[i]} \n--------------------------------------------------------------')

Similiar code snippet:

Similarity: 0.7125805616378784, 
def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between given two vectors
    :type vec1: list
    :type vec2: list
    """
    if len(vec1) != len(vec2):
        raise ValueError('The two vectors must be the same length. Got shape ' + str(len(vec1)) + ' and ' + str(len(vec2)))
    norm_a = _l2_distance(vec1)
    norm_b = _l2_distance(vec2)
    similarity = 0.0
    for (vec1_element, vec2_element) in zip(vec1, vec2):
        similarity += vec1_element * vec2_element
    similarity /= norm_a * norm_b
    return similarity 
--------------------------------------------------------------
Similarity: 0.621502161026001, 
def test_cosine_similarity(self):
    vec_a = [1, 1, 1]
    vec_b = [-1, -1, -1]
    vec_c = [1, 2, -1]
    self.assertAlmostEqual(cosine_similarity(vec_a, vec_a), 1)
    self.assertAlmostEqual(cosine_similarity(vec_a, vec_b), -1)
    self.assertAlmostEqual(cosine_similarity(vec_a, vec_c), 0.